A Neural Machine Translation (NMT) model to translate human-readable dates ("25th of June, 2009") into machine-readable dates ("2009-06-25").

An attention model, one of the most sophisticated sequence-to-sequence models will be build to carry out the task

#Packages

In [69]:
from tensorflow.keras.layers import Bidirectional, Concatenate, Permute, Dot, Input, LSTM, Multiply
from tensorflow.keras.layers import RepeatVector, Dense, Activation, Lambda
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import load_model, Model
import tensorflow.keras.backend as K
import tensorflow as tf
import numpy as np
from faker import Faker
import random
from tqdm import tqdm
from babel.dates import format_date
from nmt_utils import *
import matplotlib.pyplot as plt
%matplotlib inline

# Dataset

We will train the model on a dataset of 10,000 human readable dates and their equivalent, standardized, machine readable dates.

In [70]:
m = 10000
# dataset: a list of tuples of (human readable date, machine readable date).
# human_vocab: a python dictionary mapping all characters used in the human readable dates to an integer-valued index.
# machine_vocab: a python dictionary mapping all characters used in machine readable dates to an integer-valued index.
# Note: These indices are not necessarily consistent with human_vocab.
# inv_machine_vocab: the inverse dictionary of machine_vocab, mapping from indices back to characters.
dataset, human_vocab, machine_vocab, inv_machine_vocab = load_dataset(m)

100%|██████████| 10000/10000 [00:00<00:00, 17937.74it/s]


In [71]:
dataset[:10]

[('28 oct 1990', '1990-10-28'),
 ('1 may 1998', '1998-05-01'),
 ('wednesday april 2 2003', '2003-04-02'),
 ('29.04.99', '1999-04-29'),
 ('august 27 2008', '2008-08-27'),
 ('october 11 1991', '1991-10-11'),
 ('16 apr 1987', '1987-04-16'),
 ('march 26 2021', '2021-03-26'),
 ('monday july 18 1977', '1977-07-18'),
 ('tuesday january 24 2012', '2012-01-24')]

In [72]:
human_vocab

{' ': 0,
 '.': 1,
 '/': 2,
 '0': 3,
 '1': 4,
 '2': 5,
 '3': 6,
 '4': 7,
 '5': 8,
 '6': 9,
 '7': 10,
 '8': 11,
 '9': 12,
 '<pad>': 36,
 '<unk>': 35,
 'a': 13,
 'b': 14,
 'c': 15,
 'd': 16,
 'e': 17,
 'f': 18,
 'g': 19,
 'h': 20,
 'i': 21,
 'j': 22,
 'l': 23,
 'm': 24,
 'n': 25,
 'o': 26,
 'p': 27,
 'r': 28,
 's': 29,
 't': 30,
 'u': 31,
 'v': 32,
 'w': 33,
 'y': 34}

In [73]:
tf.config.list_physical_devices()

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]

In [74]:
Tx = 30 # maximum lenght of the human readable data
Ty = 10 # length of the output string,"YYYY-MM-DD" is 10 characters long.
X, Y, Xoh, Yoh = preprocess_data(dataset, human_vocab, machine_vocab, Tx, Ty)

print("X.shape:", X.shape)
print("Y.shape:", Y.shape)
print("Xoh.shape:", Xoh.shape)
print("Yoh.shape:", Yoh.shape)

X.shape: (10000, 30)
Y.shape: (10000, 10)
Xoh.shape: (10000, 30, 37)
Yoh.shape: (10000, 10, 11)


In [75]:
index = 0
print("Source date:", dataset[index][0])
print("Target date:", dataset[index][1])
print()
print("Source after preprocessing (indices):", X[index])
print("Target after preprocessing (indices):", Y[index])
print()
print("Source after preprocessing (one-hot):", Xoh[index])
print("Target after preprocessing (one-hot):", Yoh[index])

Source date: 28 oct 1990
Target date: 1990-10-28

Source after preprocessing (indices): [ 5 11  0 26 15 30  0  4 12 12  3 36 36 36 36 36 36 36 36 36 36 36 36 36
 36 36 36 36 36 36]
Target after preprocessing (indices): [ 2 10 10  1  0  2  1  0  3  9]

Source after preprocessing (one-hot): [[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 1.]]
Target after preprocessing (one-hot): [[0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]]


#Overview of the attention model

![](https://raw.githubusercontent.com/amanchadha/coursera-deep-learning-specialization/3a623a00267716d1695e0ce57480f9027648ad4e/C5%20-%20Sequence%20Models/Week%203/Machine%20Translation/images/attn_mechanism.png)
![](https://raw.githubusercontent.com/amanchadha/coursera-deep-learning-specialization/3a623a00267716d1695e0ce57480f9027648ad4e/C5%20-%20Sequence%20Models/Week%203/Machine%20Translation/images/attn_model.png)

In [76]:
# Defined shared layers as global variables
repeator = RepeatVector(Tx)
concatenator = Concatenate(axis=-1)
densor1 = Dense(10, activation = "tanh")
densor2 = Dense(1, activation = "relu")
activator = Activation(softmax, name='attention_weights') # We are using a custom softmax(axis = 1) loaded in this notebook
dotor = Dot(axes = 1)

In [77]:
def one_step_attention(a, s_prev):
    """
    Performs one step of attention: Outputs a context vector computed as a dot product of the attention weights
    "alphas" and the hidden states "a" of the Bi-LSTM.
    
    Arguments:
    a -- hidden state output of the Bi-LSTM, numpy-array of shape (m, Tx, 2*n_a)
    s_prev -- previous hidden state of the (post-attention) LSTM, numpy-array of shape (m, n_s)
    
    Returns:
    context -- context vector, input of the next (post-attention) LSTM cell
    """
    # Use repeator to repeat s_prev to be of shape (m, Tx, n_s) so that you can concatenate it with all hidden states "a" 
    s_prev = repeator(s_prev)
    # Use concatenator to concatenate a and s_prev on the last axis
    concat = concatenator([a,s_prev])
    # Use densor1 to propagate concat through a small fully-connected neural network to compute the "intermediate energies" variable e
    e = densor1(concat)
    # Use densor2 to propagate e through a small fully-connected neural network to compute the "energies" variable energies.
    energies = densor2(e)
    # Use "activator" on "energies" to compute the attention weights "alphas" 
    alphas = activator(energies)
    # Use dotor together with "alphas" and "a", in this order, to compute the context vector to be given to the next (post-attention) LSTM-cell
    context = dotor([alphas,a])
    
    return context

In [78]:
n_a = 32 # number of units for the pre-attention, bi-directional LSTM's hidden state 'a'
n_s = 64 # number of units for the post-attention LSTM's hidden state "s"

# this is the post attention LSTM cell.  
post_activation_LSTM_cell = LSTM(n_s, return_state = True) 
output_layer = Dense(len(machine_vocab), activation=softmax)

#Model

In [79]:
def modelf(Tx, Ty, n_a, n_s, human_vocab_size, machine_vocab_size):
    """
    Arguments:
    Tx -- length of the input sequence
    Ty -- length of the output sequence
    n_a -- hidden state size of the Bi-LSTM
    n_s -- hidden state size of the post-attention LSTM
    human_vocab_size -- size of the python dictionary "human_vocab"
    machine_vocab_size -- size of the python dictionary "machine_vocab"

    Returns:
    model -- Keras model instance
    """
    
    # Define the inputs of the model with a shape (Tx,)
    # Define s0 (initial hidden state) and c0 (initial cell state)
    # for the decoder LSTM with shape (n_s,)
    X = Input(shape=(Tx, human_vocab_size))
    s0 = Input(shape=(n_s,), name='s0')
    c0 = Input(shape=(n_s,), name='c0')
    s = s0
    c = c0
    
    # Initialize empty list of outputs
    outputs = []
    a = Bidirectional(LSTM(n_a, return_sequences=True))(X) # pre-attention Bi-LSTM
    
    for t in range(Ty):
        context = one_step_attention(a, s)

        s, _, c = post_activation_LSTM_cell(context,initial_state=[s, c])
        
        out = output_layer(s)
        
        outputs.append(out)
    model = Model(inputs=[X, s0, c0],outputs=outputs)
    return model

In [80]:
model = modelf(Tx, Ty, n_a, n_s, len(human_vocab), len(machine_vocab)) # create the model

In [81]:
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 30, 37)]     0           []                               
                                                                                                  
 s0 (InputLayer)                [(None, 64)]         0           []                               
                                                                                                  
 bidirectional_2 (Bidirectional  (None, 30, 64)      17920       ['input_3[0][0]']                
 )                                                                                                
                                                                                                  
 repeat_vector_1 (RepeatVector)  (None, 30, 64)      0           ['s0[0][0]',               

In [82]:
# compilation
opt = Adam(learning_rate=0.005, beta_1=0.9, beta_2=0.999, decay=0.01)
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

In [83]:
s0 = np.zeros((m, n_s))
c0 = np.zeros((m, n_s))
outputs = list(Yoh.swapaxes(0,1))

In [84]:
model.fit([Xoh, s0, c0], outputs, epochs=1, batch_size=100)

100/100 [==============================] - 28s 80ms/step - loss: 17.4401 - dense_5_loss: 1.2771 - dense_5_1_loss: 1.1232 - dense_5_2_loss: 1.8888 - dense_5_3_loss: 2.6730 - dense_5_4_loss: 0.7981 - dense_5_5_loss: 1.4124 - dense_5_6_loss: 2.8489 - dense_5_7_loss: 1.0304 - dense_5_8_loss: 1.7926 - dense_5_9_loss: 2.5956 - dense_5_accuracy: 0.4756 - dense_5_1_accuracy: 0.6488 - dense_5_2_accuracy: 0.2692 - dense_5_3_accuracy: 0.0785 - dense_5_4_accuracy: 0.9891 - dense_5_5_accuracy: 0.2707 - dense_5_6_accuracy: 0.0276 - dense_5_7_accuracy: 0.9604 - dense_5_8_accuracy: 0.2175 - dense_5_9_accuracy: 0.0919


In [87]:
model.load_weights('/content/models/model.h5') # model was trained longer ,so load the model with trained weights

#Prediction new data

In [88]:
EXAMPLES = ['3 May 1979', '5 April 09', '21th of August 2016', 'Tue 10 Jul 2007', 'Saturday May 9 2018', 'March 3 2001', 'March 3rd 2001', '1 March 2001']
s00 = np.zeros((1, n_s))
c00 = np.zeros((1, n_s))
for example in EXAMPLES:
    source = string_to_int(example, Tx, human_vocab)
    #print(source)
    source = np.array(list(map(lambda x: to_categorical(x, num_classes=len(human_vocab)), source))).swapaxes(0,1)
    source = np.swapaxes(source, 0, 1)
    source = np.expand_dims(source, axis=0)
    prediction = model.predict([source, s00, c00])
    prediction = np.argmax(prediction, axis = -1)
    output = [inv_machine_vocab[int(i)] for i in prediction]
    print("source:", example)
    print("output:", ''.join(output),"\n")

source: 3 May 1979
output: 1979-05-33 

source: 5 April 09
output: 2009-04-05 

source: 21th of August 2016
output: 2016-08-20 

source: Tue 10 Jul 2007
output: 2007-07-10 

source: Saturday May 9 2018
output: 2018-05-09 

source: March 3 2001
output: 2001-03-03 

source: March 3rd 2001
output: 2001-03-03 

source: 1 March 2001
output: 2001-03-01 



In [89]:
def translate_date(sentence):
    source = string_to_int(sentence, Tx, human_vocab)
    source = np.array(list(map(lambda x: to_categorical(x, num_classes=len(human_vocab)), source))).swapaxes(0,1)
    source = np.swapaxes(source, 0, 1)
    source = np.expand_dims(source, axis=0)
    prediction = model.predict([source, s00, c00])
    prediction = np.argmax(prediction, axis = -1)
    output = [inv_machine_vocab[int(i)] for i in prediction]
    print("source:", sentence)
    print("output:", ''.join(output),"\n")
example = "4th of july 2001"
translate_date(example)

source: 4th of july 2001
output: 2001-07-04 

